In [ ]:
np.sum(np.diff(x) * (y[:-1] + np.diff(y)/2))

In [14]:
import numpy as np

In [63]:
class Preferences(object):
    def __init__(self):
        self.end = 0
        self.endvalue = 0
        self.id = 0
        self.start = 0
        self.startvalue = 0

In [64]:
#row is agent, column is slice
preferences = [[Preferences() for i in range(3)] for j in range(3)]

In [133]:
preferences[0][0].end = 1
preferences[0][0].endvalue = 3.7
preferences[0][0].id = 1
preferences[0][0].start = 0
preferences[0][0].startvalue = 5.8

In [134]:
preferences[0][1].end = 2
preferences[0][1].endvalue = 5.5
preferences[0][1].id = 2
preferences[0][1].start = 1
preferences[0][1].startvalue = 5.5

In [135]:
preferences[0][2].end = 3
preferences[0][2].endvalue = 7
preferences[0][2].id = 3
preferences[0][2].start = 2
preferences[0][2].startvalue = 1.6

In [136]:
preferences[1][0].end = 1
preferences[1][0].endvalue = 7.3
preferences[1][0].id = 4
preferences[1][0].start = 0
preferences[1][0].startvalue = 3

In [137]:
preferences[1][1].end = 2
preferences[1][1].endvalue = 7.5
preferences[1][1].id = 5
preferences[1][1].start = 1
preferences[1][1].startvalue = 3

In [138]:
preferences[1][2].end = 3
preferences[1][2].endvalue = 0
preferences[1][2].id = 6
preferences[1][2].start = 2
preferences[1][2].startvalue = 10

In [139]:
preferences[2][0].end = 1
preferences[2][0].endvalue = 4.3
preferences[2][0].id = 7
preferences[2][0].start = 0
preferences[2][0].startvalue = 9.4

In [140]:
preferences[2][1].end = 3
preferences[2][1].endvalue = 7.6
preferences[2][1].id = 8
preferences[2][1].start = 1
preferences[2][1].startvalue = 7.6

In [141]:
preferences[2][2] = None

In [142]:
cakeSize = 3

In [143]:
#prefs, normalizationConstant = oneLipschitz(preferences, cakeSize)
preferences = oneLipschitz(preferences, cakeSize)

In [ ]:
epsilion = 0.003

Preprocessing

Hungriness:

In [ ]:
def branzeiNisan(prefs, cakeSize, epsilon):
    preferences = oneLipschitz(prefs, cakeSize)
    alphaLowerBound = computeEquipartitionFourAgents(prefs, epsilon)
    if checkEnvyFree(prefs, alphaLowerBound) == True:
        return division(prefs, alphaLowerBound)
    alphaUpperBound = 1
    while abs(alphaUpperBound - alphaLowerBound) > ((epsilon/10)**4)/12:
        alpha = (alphaLowerBound + alphaUpperBound) / 2
        if checkConditions(prefs, alpha) == True:
            alphaLowerBound = alpha
        else:
            alphaUpperBound = alpha
    return division(prefs, alphaLowerBound)
        

In [ ]:
def computeEquipartitionThreeAgents(prefs, epsilon):
    leftCutBounds = findLeftCutBounds(prefs, epsilon)
    rightCutBounds = findRightCutBounds(prefs, epsilon)

In [ ]:
def findLeftCutBounds(prefs, epsilon):
    leftCutUpperBound = 1
    leftCutLowerBound = 0
    middleSegmentValue = 2
    while abs(leftCutUpperBound - leftCutLowerBound) - epsilon > 1e-10:
        leftCut = (leftCutUpperBound + leftCutLowerBound)/2
        leftSegmentValue = valueQuery(0, prefs, 0, leftCut)
        rightCutUpperBound = 1
        rightCutLowerBound = leftCut
        while abs(middleSegmentValue - leftSegmentValue) > 1e-15:
            rightCut = (rightCutUpperBound + rightCutLowerBound)/2
            middleSegmentValue = valueQuery(0, prefs, leftCut, rightCut)
            if middleSegmentValue < leftSegmentValue:
                rightCutLowerBound = rightCut
            if middleSegmentValue > leftSegmentValue:
                rightCutUpperBound = rightCut
        rightSegmentValue = valueQuery(0, prefs, rightCut, 1)
        if rightSegmentValue > leftSegmentValue:
            leftCutLowerBound = LeftCut
        if rightSegmentValue < leftSegmentValue:
            leftCutUpperBound = LeftCut
    return leftCutLowerBound, leftCutUpperBound
            

In [105]:
#Remember to scale by L at end.
def oneLipschitz(prefs, cakeSize):
    prefs = changeBounds(prefs, cakeSize)
    #prefs, normalizationConstants = normalization(prefs)
    prefs = normalization(prefs)
    return prefs#, normalizationConstants

In [27]:
def changeBounds(prefs, cakeSize):
    agents = np.shape(prefs)[0]
    segments = np.shape(prefs)[1]
    for i in range(agents):
        for j in range(segments):
            if prefs[i][j] is not None:
                prefs[i][j].start /= cakeSize
                prefs[i][j].end /= cakeSize
    return prefs
        

In [79]:
'''
Ensures that each agents values the
'''
def normalization(prefs):
    agents = np.shape(prefs)[0]
    segments = np.shape(prefs)[1]
    normalizationConstants = np.zeros(agents)
    for i in range(agents):
        normalizationConstants[i] = valueQueryInitial(i, prefs, 0, 1)
        for j in range(segments):
            if prefs[i][j] is not None:
                prefs[i][j].startvalue /= normalizationConstants[i]
                prefs[i][j].endvalue /= normalizationConstants[i]
    return prefs#, normalizationConstants
        

In [153]:
'''
Linear interpolation on epsilon grid.
'''
def valueQuery(agent, prefs, start, end, epsilon):
    startUpperBound = (start // epsilon) * epsilon + epsilon
    endLowerBound = (end // epsilon) * epsilon
    
    if startUpperBound - start >= end - endLowerBound:
        return valueQueryVariantOne(agent, prefs, start, end, epsilon)
    if startUpperBound - start < end - endLowerBound:
        return valueQueryVariantTwo(agent, prefs, start, end, epsilon)

0.9999999999999999

In [154]:
def valueQueryVariantOne(agent, prefs, start, end, epsilon):
    startLowerBound = (start // epsilon) * epsilon
    startUpperBound = startLowerBound + epsilon
    endLowerBound = (end // epsilon) * epsilon
    endUpperBound = endLowerBound + epsilon

    queryOne = valueQueryHungry(agent, prefs, startLowerBound, endLowerBound)
    queryTwo = valueQueryHungry(agent, prefs, startLowerBound, endUpperBound)
    queryThree = valueQueryHungry(agent, prefs, startUpperBound, endLowerBound)
    
    ComponentOne = (((startUpperBound - start) - (end - endLowerBound)) / epsilon) * \
                    queryOne
    ComponentTwo = ((end - endLowerBound) / epsilon) * queryTwo
    ComponentThree = ((start - startLowerBound) / epsilon) * queryThree

    return componentOne + componentTwo + componentThree
                    
    

In [155]:
def valueQueryVariantTwo(agent, prefs, start, end, epsilon):
    startLowerBound = (start // epsilon) * epsilon
    startUpperBound = startLowerBound + epsilon
    endLowerBound = (end // epsilon) * epsilon
    endUpperBound = endLowerBound + epsilon

    queryOne = valueQueryHungry(agent, prefs, startUpperBound, endUpperBound)
    queryTwo = valueQueryHungry(agent, prefs, startLowerBound, endUpperBound)
    queryThree = valueQueryHungry(agent, prefs, startUpperBound, endLowerBound)
    
    ComponentOne = (((end - endLowerBound) - (startUpperBound - start)) / epsilon) * \
                    queryOne
    ComponentTwo = ((startUpperBound - start) / epsilon) * queryTwo
    ComponentThree = ((endUpperBound - end) / epsilon) * queryThree

    return componentOne + componentTwo + componentThree
                    
    

In [156]:
def valueQueryHungry(agent, prefs, start, end, epsilon):
    initialValue = valueQueryInitial(agent, prefs, start, end)
    value = initialValue/2 + epsilon * (end - start)
    return value

In [158]:
def valueQueryInitial(agent, prefs, start, end):
    if end <= start:
        value = 0
    else:
        value = oneSidedQuery(agent, prefs, end) - oneSidedQuery(agent, prefs, start)
    return value
    

In [41]:
def oneSidedQuery(agent, prefs, end):
    segments = np.size(prefs[agent])
    value = 0
    for i in range(segments):
        if end > prefs[agent][i].end:
            value += findFullValue(prefs[agent][i])
        if end <= prefs[agent][i].end:
            value += findPartialValue(prefs[agent][i], end)
            return value

In [43]:
def findFullValue(segment):
    segmentWidth = segment.end - segment.start
    if segment.startvalue == segment.endvalue:
        value = segment.startvalue * segmentWidth
        return value
    if segment.startvalue != segment.endvalue:
        value = 0.5 * (segment.startvalue + segment.endvalue) * segmentWidth
        return value

In [33]:
def findPartialValue(segment, partialSegmentEnd):
    segmentWidth = partialSegmentEnd - segment.start
    partialSegmentEndValue = interpolate(segment, partialSegmentEnd)
    if segment.startvalue == segment.endvalue:
        value = segment.startvalue * segmentWidth
        return value
    if segment.startvalue != segment.endvalue:
        value = 0.5 * (segment.startvalue + partialSegmentEndValue) * segmentWidth
        return value

In [31]:
'''
Uses the linear interpolation formula to find the associated y-coordinate for a 
given x-coordinate on a line between two points with known x and y coordinates.
'''
def interpolate(segment, xCoordinate):
    if segment.startvalue == segment.endvalue:
        yCoordinate = segment.startvalue
        return yCoordinate
    if segment.startvalue != segment.endvalue:
        yCoordinate = ((segment.startvalue * (segment.end - xCoordinate) +
                       segment.endvalue * (xCoordinate - segment.start)) / 
                       (segment.end - segment.start))                
        return yCoordinate

In [53]:
interpolate(preferences[0][1], 1.3)

5.5

In [52]:
preferences[0][1].endvalue

5.5

In [42]:
np.size(preferences[1])

3

In [ ]:
def removeZeros(prefs):
    if prefs.end == 0:
        prefs.end = 0.00001
    if prefs

In [41]:
if preferences[2][1] is not None:
    print("not none")

not none
